# Day 2 - Timeseries

Today, we will practice manipulating timeseries data. We will use lidar measurements of wind speeds as our toy dataset. These measurements were gathered off the East coast in an area where companies are planning to build wind turbines. We will use these measurements to answer questions like "what month has the strongest average winds" and "how often are winds so weak that they don't spin the turbine".

In [ ]:
### Import libraries
from datetime import datetime  # for some datetime manipulations
import numpy as np # for data storage and math
import pandas as pd # for data storage and timeseries math
import matplotlib.pyplot as plt # for plotting

# Read in data
Our lidar measurements are saved as `.csv`'s, which stands for Comma Separated Values. You can think of `.csv` files as spreadsheets where data within a row is seperated with a comma. You can double click on `lidar_winds.csv` in the sidebar and glimpse the data. Tabular data shows up all the time in the geosciences, and one place it is especially common is observational timeseries.

`pandas` is a library that is specifically designed to read and analyze tabular data. We will start off by reading our toy dataset of lidar winds into a `pandas` DataFrame.

In [ ]:
### Read tabluar data into a pandas DataFrame
df = pd.read_csv("lidar_winds.csv")

We can get a quick idea of what data inside our DataFrame looks like by looking at the first 5 rows with the `.head()` command.

In [ ]:
df.head()

and you can look at the end of the DataFram with the `.tail()` command.

In [ ]:
df.tail()

`df` has 11 **columns**. One has timestamps of wind speed measurements, and the rest are wind speed measurements at heights between 18 m and 198 m above the surface. We can isolate just one column of the DataFrame by using `df["wspd18"]`, for example

In [ ]:
df['wspd18m']

or looking at timestamp values, we run

In [ ]:
df['timestamp']

# Preprocessing

Before doing any sort of analysis (for example, calculating things like average wind speeds), you often need to preprocess data. `pandas` is very powerful at doing timeseries analysis, but in order to do timeseries analysis, we must reformat the data in a way that is friendly to timeseries analysis. We are going to do two things: reformat the data and change the index.

The last row of the above outputs tells us that the data in `wspd18m` is `float64` (a decimal number) and the data in `timestamp` is `object` (which usually means a string). We need to tell `pandas` that the `timestamp` column actually contains timestamps, not just any old regular strings. To do that, we will run the following line

In [ ]:
pd.to_datetime(df['timestamp'])

You can now see that the `dtype` in the last line is now `datetime64`. In `pd.to_datetime()`, `pandas` reads though all the data and guessts its format. If you know the format ahead of time, you can specify the format and the data conversion will run quicker. For example, the conversion for the above code would read `pd.to_datetime(df['timestamp'], format='%m-%d-%Y %H:%M')`. [More on conversion codes here.](https://strftime.org/)

### Common pitfall
Above, we ran a line of code that converted the `object` format data to `datetime64` formatted data. So, if we rerun `df['timestamp']`, we should see that our data is now `datetime64` data, right?

In [ ]:
df['timestamp']

But in actuality, it's still the `object` datatype. What gives? 

In `pandas`, **you often need to explicitly state that you want to replace the data in your DataFrame**. In order to replace the data in `timestamp`, run

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

Now, when we look at the data in the `timestamp` column, we see that it is indeed `datetime64`.

In [ ]:
df['timestamp']

Now that `pandas` knows that we are working with timeseries data, we need to replace the index with these timestamps. In the above code outputs, you see the numbers `0, 1, 2, ...` to the left of all the data, one for every row. This is the **index**. When reading in data via `pd.read_csv()`, we didn't specify the value of the index, so `pandas` set the value of the index to its default value of `0, 1, 2, ...`. We want to replace this default index with the timeseries index. To do this, run `df = df.set_index("<column name>")`.

In [ ]:
df = df.set_index('timestamp')

After running this command, we are done preprocessing. You can output the value of `df` and verify that it looks slightly different now than when we started.

In [ ]:
df

### By the way: selecting data by row
We earlier isolated data by column. We can also isolate data by row. There are two ways to do this, by "position" and by "row value". If you want to access the data in the 3rd row by "position", run `df.iloc[2]` (remember Python's zero-indexing). If you want to access data by "value", run `df.loc[datetime(2019, 8, 12, 0, 20)]`. We need to be verbose and state `datetime(2019, 8, 12, 0, 20)` because our data is the `datetime64` data type.

# Processing the data

Now that our data is nicely formatted, we can begin to ask questions. For example, what is the average wind speed at every height? To get the average wind speed across all timesteps, run `df.mean()`.

In [ ]:
df.mean()

Instead of selecting all timestamps, we can also subselect data by characteristics. For example, we can subselect all data in September. Note, this grabs data in *both* September 2019 *and* September 2020.

In [ ]:
df[df.index.month == 9]

You can also grab data from *only* September 2020.

In [ ]:
df[(df.index.month == 9) & (df.index.year == 2020)]

We can find the average wind speeds in September 2020 by running `.mean()` on the data that we subselect.

In [ ]:
df[(df.index.month == 9) & (df.index.year == 2020)].mean()

### Visualizing the data

So far, we have been looking at average wind speed behavior. How do winds behave throughout time? Let's plot 138 m winds.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,3))

ax.plot(df.index, df['wspd138m'])

ax.set_ylabel("Wind Speed [m/s]", fontsize=12)
ax.set_xlabel("Date", fontsize=12)

plt.show()

We can see that instantaneous winds can be as strong as 30 m/s or as weak as 0 m/s. Let's smooth this time series by averaging winds every week. 

In [ ]:
weekly_winds = df.resample("W").mean()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,3))

ax.plot(df.index, df['wspd138m'])
ax.plot(weekly_winds.index, weekly_winds['wspd138m'], color='orange')

ax.set_ylabel("Wind Speed [m/s]", fontsize=12)
ax.set_xlabel("Date", fontsize=12)

plt.show()

# Challenge: Compare data from lidar and a nearby buoy
The lidar is located at 39.9695, -72.160. We can grab measurements from a meteorological buoy, courtesy of [NOAA National Data Buoy Center](https://www.ndbc.noaa.gov/). [Buoy 44025](https://www.ndbc.noaa.gov/station_history.php?station=44025) is at 40.251, -73.164. 

Column units:
* WDIR: degT
* WSPD: m/s
* GST: m/s
* WHVT: m
* DPD: sec
* APD: sec
* MWD: degT
* PRES: hPa
* ATMP: degC
* WTMP: degC
* DEWP: degC
* VIS: mi
* TIDE: ft

In [ ]:
df_buoy = pd.read_csv('buoy_data.csv')

In [ ]:
df_buoy